In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
import torch
import math
import re
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from tokenizers import ByteLevelBPETokenizer
from tqdm import tqdm
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df = pd.read_csv("/kaggle/input/poetry/poetry.csv")

In [3]:
df = df.iloc[:len(df) // 100]

In [4]:
pd.set_option('display.max_colwidth', None)
df.tail()

,Unnamed: 0,Title,Poem,Poet,Tags
133,133,\r\r\n Book Review: The Mountain Lion by Jean Stafford\r\r\n,"\r\r\nSpoiler alert: in this all-but-forgotten\r\r\nmasterwork, Jean Stafford—who was once\r\r\nwidely regarded as the leading novelist\r\r\n \r\r\nof her generation, and who wrote\r\r\nthis perverse, short,\r\r\nlyrical novel, her second, during\r\r\n \r\r\nthe flailing failings\r\r\nof her marriage to my hero\r\r\nRobert Lowell—kills\r\r\n \r\r\nMolly, her child-alter ego,\r\r\na girl too unloved and unloving\r\r\nto survive puberty, too\r\r\n \r\r\npure and awful—like Stafford, who died\r\r\npickled and childish three\r\r\ndecades later after winning\r\r\n \r\r\nthe Pulitzer with her devastating,\r\r\nhurtfully compassionate Collected Stories—for this or any other world,\r\r\n \r\r\nespecially the necessarily\r\r\nallegorical one of fiction.\r\r\nI am broken now, hopeless; hope\r\r\n \r\r\nis proved by this book to be\r\r\na contrivance. I have just\r\r\nread the last pages in which\r\r\n \r\r\nMolly’s brother, Ralph—who,\r\r\nto live, cannot love\r\r\neither, has no spare love—shoots\r\r\n \r\r\nher, aiming for the wild mountain\r\r\nlion whose stuffed corpse\r\r\nwas to be the triumph\r\r\n \r\r\nof his new manhood. I don’t\r\r\nhate Ralph—how can I, a boy,\r\r\nmistaken, like me? And can I hate\r\r\n \r\r\nMolly, who so needed Ralph\r\r\nand everyone, still young enough to savor\r\r\nthe bittersweet of her anger?\r\r\n \r\r\nWhat about Stafford, who hurt\r\r\nherself, all our selves, with\r\r\nthis ending, her classic tragedy, writing,\r\r\n \r\r\ndecades later, Poor old Molly! I loved her dearly and I hope she rests in peace.\r\r\n \r\r\nFuck insight and analysis:\r\r\nmy heart is shot. Why\r\r\ndid she have to die? Why does\r\r\n \r\r\nanyone? Why do you, do I?\r\r\nBecause of what Ralph was\r\r\nfeeling just before he accidentally\r\r\n \r\r\nslaughtered the future? This book\r\r\nmust have ravaged the already\r\r\nsleepless poet Gregory Orr,\r\r\n \r\r\nwho shot his brother, too, and\r\r\nsuffers that endless error\r\r\nin poetry and prose. And because Molly\r\r\n \r\r\nrefused everything, she stood between\r\r\nRalph and tomorrow. But he grew, he\r\r\nchanged. Confused? Read\r\r\n \r\r\nthe book. In novels\r\r\npeople die because of what they feel.\r\r\nIn life, people die when\r\r\n \r\r\ntheir bodies conk out,\r\r\nexhausted machines that living\r\r\nexpends. And what\r\r\n \r\r\nhappens when people feel\r\r\ntheir feelings in life?\r\r\nNothing? Anything? Brenda,\r\r\n \r\r\ndear Brenda, my love, nothing\r\r\nhappens, I’m afraid. I’m sorry. And afraid.\r\r\nA small breeze born in the heart\r\r\n \r\r\ngently bends a blade of grass\r\r\nand no one hears a word.\r\r\nNo one reads Stafford anymore—I asked\r\r\n \r\r\non Facebook. Stafford died, her\r\r\nlegacy gently dispatched\r\r\ninto the low air. O, life\r\r\n \r\r\nis terrible, literature\r\r\nridiculous. Stafford’s prose,\r\r\nteeming and rich as loam,\r\r\n \r\r\ncould take Famous Franzen’s\r\r\nfor a walk, feed it biscuits.\r\r\nBut who cares? Who remembers?\r\r\n \r\r\nO, to have been Jean Stafford,\r\r\nin the past I idealize, when the world\r\r\nwas less self-conscious, less\r\r\n \r\r\nprecise. I could be\r\r\ndead already, warmish\r\r\nbeneath a blanket of dust. Joyful\r\r\n \r\r\nare the faded, the once-greats\r\r\nwhose afterlives slipped out\r\r\na hole in posterity's pocket:\r\r\n \r\r\nthey are loved poignantly by\r\r\na needy few. O, to be kept\r\r\ncozy in the bosoms of those\r\r\n \r\r\ndesperate and proud, forgotten\r\r\nfor all the good I do. Love\r\r\nis sunlight streaming unevenly\r\r\n \r\r\nthrough the canopy of leaves\r\r\noverhead. We can only grow\r\r\nin the brighter patches below, fading\r\r\n \r\r\nwhere light is thin. Molly,\r\r\nwe are with you, nowhere and gone.\r\r\nMostly we are forgotten, too.\r\r\n \r\r\n",Craig Morgan Teicher,"Coming of Age,Death,Reading & Books"
134,134,\r\r\n Small Shame Blues\r\r\n,\r\r\nI live with the s

In [5]:
df.isna().sum()

Unnamed: 0     0
Title          0
Poem           0
Poet           0
Tags          19
dtype: int64

In [6]:
df = df.drop(['Unnamed: 0', 'Tags', 'Poet', 'Title'], axis=1)

In [7]:
# df['Title'] = df['Title'].str.replace('\r', '')
df['Poem'] = df['Poem'].str.replace('\r', '')
# df['Title'] = df['Title'].str.replace('\n', '')
df['Poem'] = df['Poem'].str.strip('\n')

In [8]:
df.tail()

,Poem
133,"Spoiler alert: in this all-but-forgotten\nmasterwork, Jean Stafford—who was once\nwidely regarded as the leading novelist\n \nof her generation, and who wrote\nthis perverse, short,\nlyrical novel, her second, during\n \nthe flailing failings\nof her marriage to my hero\nRobert Lowell—kills\n \nMolly, her child-alter ego,\na girl too unloved and unloving\nto survive puberty, too\n \npure and awful—like Stafford, who died\npickled and childish three\ndecades later after winning\n \nthe Pulitzer with her devastating,\nhurtfully compassionate Collected Stories—for this or any other world,\n \nespecially the necessarily\nallegorical one of fiction.\nI am broken now, hopeless; hope\n \nis proved by this book to be\na contrivance. I have just\nread the last pages in which\n \nMolly’s brother, Ralph—who,\nto live, cannot love\neither, has no spare love—shoots\n \nher, aiming for the wild mountain\nlion whose stuffed corpse\nwas to be the triumph\n \nof his new manhood. I don’t\nhate Ralph—how can I, a boy,\nmistaken, like me? And can I hate\n \nMolly, who so needed Ralph\nand everyone, still young enough to savor\nthe bittersweet of her anger?\n \nWhat about Stafford, who hurt\nherself, all our selves, with\nthis ending, her classic tragedy, writing,\n \ndecades later, Poor old Molly! I loved her dearly and I hope she rests in peace.\n \nFuck insight and analysis:\nmy heart is shot. Why\ndid she have to die? Why does\n \nanyone? Why do you, do I?\nBecause of what Ralph was\nfeeling just before he accidentally\n \nslaughtered the future? This book\nmust have ravaged the already\nsleepless poet Gregory Orr,\n \nwho shot his brother, too, and\nsuffers that endless error\nin poetry and prose. And because Molly\n \nrefused everything, she stood between\nRalph and tomorrow. But he grew, he\nchanged. Confused? Read\n \nthe book. In novels\npeople die because of what they feel.\nIn life, people die when\n \ntheir bodies conk out,\nexhausted machines that living\nexpends. And what\n \nhappens when people feel\ntheir feelings in life?\nNothing? Anything? Brenda,\n \ndear Brenda, my love, nothing\nhappens, I’m afraid. I’m sorry. And afraid.\nA small breeze born in the heart\n \ngently bends a blade of grass\nand no one hears a word.\nNo one reads Stafford anymore—I asked\n \non Facebook. Stafford died, her\nlegacy gently dispatched\ninto the low air. O, life\n \nis terrible, literature\nridiculous. Stafford’s prose,\nteeming and rich as loam,\n \ncould take Famous Franzen’s\nfor a walk, feed it biscuits.\nBut who cares? Who remembers?\n \nO, to have been Jean Stafford,\nin the past I idealize, when the world\nwas less self-conscious, less\n \nprecise. I could be\ndead already, warmish\nbeneath a blanket of dust. Joyful\n \nare the faded, the once-greats\nwhose afterlives slipped out\na hole in posterity's pocket:\n \nthey are loved poignantly by\na needy few. O, to be kept\ncozy in the bosoms of those\n \ndesperate and proud, forgotten\nfor all the good I do. Love\nis sunlight streaming unevenly\n \nthrough the canopy of leaves\noverhead. We can only grow\nin the brighter patches below, fading\n \nwhere light is thin. Molly,\nwe are with you, nowhere and gone.\nMostly we are forgotten, too.\n"
134,I live with the small shame\nof not knowing the multiple names for blue\nto describe the nightsky over New Mexico\nto describe the light in my lover’s eyes.\n \nIt is a small shame that grows.\n \nI live with the small shame\nwhich resides in the absences of my speech\nas I pause to search for the word in Spanish\nto translate a poem to my Father\nwho sits there waiting\nwho scans my eyes to see\nwhat I cannot fully describe\nwho waits for the word from me\nthe word that escapes me in the moment\nthe word I fear has never resided within me.\n \nIt is a small shame that grows\nwhen indigo and cerulean are merely azul\nand not añil and cerúleo.\n
135,Let us praise the immigrant\nwho leaves the tropics\nand arrives in Chicago\nin the dead of w

In [9]:
# Lower case everything
# df['Title'] = df['Title'].str.lower()
df['Poem'] = df['Poem'].str.lower()

# Remove apostrophes and join the parts
# df['Title'] = df['Title'].str.replace("'", "", regex=False).str.replace("’", "", regex=False)
df['Poem'] = df['Poem'].str.replace("'", "", regex=False).str.replace("’", "", regex=False)

In [10]:
# Uncomment only when including encoder, until then let it be commented off
# Filter out rows where both 'Title' and 'Poem' have no alphabetic characters

# df = df[df.apply(lambda x: any(c.isalpha() for c in x['Title']) and
#                             any(c.isalpha() for c in x['Poem']), axis=1)]

# Filter out rows with no value (its not NaN its '' in this dataset)
df = df[df.apply(lambda x: any(c.isalpha() for c in x['Poem']), axis=1)]

In [11]:
# Make newline a special token
# df['Title'] = 'START ' + df['Title'] + ' END'
df['Poem'] = 'START ' + df['Poem'] + ' END'

df['Poem'] = df['Poem'].apply(lambda x: re.sub(r'\n', ' N ', x))

In [12]:
def remove_numeric_tokens(text):
    tokens = word_tokenize(text)
    # Filter tokens to exclude any that contain digits
    filtered_tokens = [token for token in tokens if not re.search(r'\d', token)]
    processed_text = ' '.join(filtered_tokens)
    return processed_text

df['Poem'] = df['Poem'].apply(remove_numeric_tokens)

In [13]:
df.tail()

,Poem
133,"START spoiler alert : in this all-but-forgotten N masterwork , jean stafford—who was once N widely regarded as the leading novelist N N of her generation , and who wrote N this perverse , short , N lyrical novel , her second , during N N the flailing failings N of her marriage to my hero N robert lowell—kills N N molly , her child-alter ego , N a girl too unloved and unloving N to survive puberty , too N N pure and awful—like stafford , who died N pickled and childish three N decades later after winning N N the pulitzer with her devastating , N hurtfully compassionate collected stories—for this or any other world , N N especially the necessarily N allegorical one of fiction . N i am broken now , hopeless ; hope N N is proved by this book to be N a contrivance . i have just N read the last pages in which N N mollys brother , ralph—who , N to live , can not love N either , has no spare love—shoots N N her , aiming for the wild mountain N lion whose stuffed corpse N was to be the triumph N N of his new manhood . i dont N hate ralph—how can i , a boy , N mistaken , like me ? and can i hate N N molly , who so needed ralph N and everyone , still young enough to savor N the bittersweet of her anger ? N N what about stafford , who hurt N herself , all our selves , with N this ending , her classic tragedy , writing , N N decades later , poor old molly ! i loved her dearly and i hope she rests in peace . N N fuck insight and analysis : N my heart is shot . why N did she have to die ? why does N N anyone ? why do you , do i ? N because of what ralph was N feeling just before he accidentally N N slaughtered the future ? this book N must have ravaged the already N sleepless poet gregory orr , N N who shot his brother , too , and N suffers that endless error N in poetry and prose . and because molly N N refused everything , she stood between N ralph and tomorrow . but he grew , he N changed . confused ? read N N the book . in novels N people die because of what they feel . N in life , people die when N N their bodies conk out , N exhausted machines that living N expends . and what N N happens when people feel N their feelings in life ? N nothing ? anything ? brenda , N N dear brenda , my love , nothing N happens , im afraid . im sorry . and afraid . N a small breeze born in the heart N N gently bends a blade of grass N and no one hears a word . N no one reads stafford anymore—i asked N N on facebook . stafford died , her N legacy gently dispatched N into the low air . o , life N N is terrible , literature N ridiculous . staffords prose , N teeming and rich as loam , N N could take famous franzens N for a walk , feed it biscuits . N but who cares ? who remembers ? N N o , to have been jean stafford , N in the past i idealize , when the world N was less self-conscious , less N N precise . i could be N dead already , warmish N beneath a blanket of dust . joyful N N are the faded , the once-greats N whose afterlives slipped out N a hole in posteritys pocket : N N they are loved poignantly by N a needy few . o , to be kept N cozy in the bosoms of those N N desperate and proud , forgotten N for all the good i do . love N is sunlight streaming unevenly N N through the canopy of leaves N overhead . we can only grow N in the brighter patches below , fading N N where light is thin . molly , N we are with you , nowhere and gone . N mostly we are forgotten , too . N END"
134,START i live with the small shame N of not knowing the multiple names for blue N to describe the nightsky over new mexico N to describe the light in my lovers eyes . N N it is a small shame that grows . N N i live with the small shame N which resides in the absences of my speech N as i pause to search for the word in spanish N to translate a poem to my father N who sits there waiting N who scans my eyes to see N what i can not fully describe N who waits for the word from me N the word that escapes me in the moment N the word i fear has never resided within me . N N it is

In [14]:
df.isna().sum()

Poem    0
dtype: int64

In [15]:
df.size

138

In [16]:
tokenizer = ByteLevelBPETokenizer()

# Saving poems to a file (required for tokenizer training)
poem_file = "poems.txt"
with open(poem_file, "w", encoding='utf-8') as file:  # Specify UTF-8 encoding
    file.write('\n'.join(df['Poem']))

# Train the tokenizer on the poems
tokenizer.train(files=[poem_file], vocab_size=30000, min_frequency=1, special_tokens=["START", "END", "N", "UNK", "PAD"])

In [17]:
# Tokenize poems and store the token IDs
df['Tokenized_Poem'] = df['Poem'].apply(lambda poem: tokenizer.encode(poem).ids)
df['Token_Length'] = df['Tokenized_Poem'].apply(len)

# Find the longest lengths and sort them
sorted_lengths = sorted(df['Token_Length'], reverse=True)
top_15_lengths = sorted_lengths[:15]
print("Top 15 longest tokenized poem lengths:", top_15_lengths)

max_length = 1000

# Ensure END token is known or defined
END_token_id = tokenizer.token_to_id("END")

# Function to trim poems and append END token
def trim_and_append_end(poem_tokens):
    if len(poem_tokens) > max_length:
        return poem_tokens[:max_length] + [END_token_id]
    return poem_tokens

# Apply trimming and END token appending
df['Tokenized_Poem'] = df['Tokenized_Poem'].apply(trim_and_append_end)

# Update the 'Token_Length' to reflect the post-operation lengths
df['Token_Length'] = df['Tokenized_Poem'].apply(len)

# Appoint longest_length as the longest poem length after all operations
longest_length = max(df['Token_Length'])
print("Longest length after operations:", longest_length)

Top 15 longest tokenized poem lengths: [1353, 1142, 911, 791, 790, 761, 676, 624, 622, 617, 613, 606, 554, 547, 544]
Longest length after operations: 1001


In [18]:
vocab_size = tokenizer.get_vocab_size()  # Number of unique words in the vocabulary
embedding_dim = 64
batch_size = 16 # Increase batch size if resources allow as it bring stabilization, 1 is very noisy
learning_rate = 0.01 # changed lr because maybe the embedding dim is too low and lr is too high so gradient is just bouncing around and not learning much
heads = 4
epochs = 15
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
for poem in df['Tokenized_Poem']:
    print(poem)
    print(len(poem))
    break

[0, 1011, 2712, 268, 9528, 268, 224, 2, 7880, 6598, 268, 9503, 2997, 224, 2, 635, 379, 980, 341, 6196, 413, 4958, 224, 2, 6698, 268, 5452, 4337, 270, 224, 2, 8588, 2730, 268, 9068, 268, 224, 2, 1011, 6396, 268, 3863, 7166, 224, 2, 635, 379, 980, 573, 3446, 7313, 270, 224, 2, 415, 3018, 270, 415, 1407, 270, 224, 2, 6499, 7717, 268, 6022, 287, 5825, 268, 1549, 224, 2, 4359, 287, 7341, 6935, 224, 2, 635, 379, 980, 341, 4485, 413, 1048, 5496, 3884, 224, 2, 1248, 268, 3884, 1300, 270, 224, 2, 4595, 1797, 268, 1704, 2590, 268, 224, 2, 2143, 7878, 7169, 224, 2, 635, 379, 980, 341, 3815, 270, 447, 8633, 224, 2, 294, 731, 270, 224, 2, 8859, 6690, 268, 9346, 268, 224, 2, 6287, 4031, 268, 6375, 1586, 224, 2, 635, 379, 980, 268, 447, 5567, 17, 587, 270, 447, 687, 224, 2, 263, 7108, 287, 779, 270, 224, 1]
158


In [20]:
def preprocessing(df, pad_token, batch_size):
    input_tensor = []
    target_tensor = []
    for poem in df['Tokenized_Poem']:
        for i in range(len(poem)-1):
            input_tensor.append(torch.tensor(poem[:i+1]))
            target_tensor.append(poem[i+1])
    
    
    input_tensor = torch.nn.utils.rnn.pad_sequence(input_tensor, padding_value=pad_token, batch_first=True)
    target_tensor = torch.tensor(target_tensor)
    
    return input_tensor, target_tensor

pad_token = tokenizer.token_to_id("PAD")         
input_tensor, target_tensor = preprocessing(df, pad_token, batch_size)

In [21]:
target_tensor.shape

torch.Size([40218])

In [22]:
input_tensor.shape

torch.Size([40218, 1000])

In [23]:
target_tensor[40]

tensor(3863)

In [24]:
input_tensor[40]

tensor([   0, 1011, 2712,  268, 9528,  268,  224,    2, 7880, 6598,  268, 9503,
        2997,  224,    2,  635,  379,  980,  341, 6196,  413, 4958,  224,    2,
        6698,  268, 5452, 4337,  270,  224,    2, 8588, 2730,  268, 9068,  268,
         224,    2, 1011, 6396,  268,    4,    4,    4,    4,    4,    4,    4,
           4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
           4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
           4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
           4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
           4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
           4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
           4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
           4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
           4,    4,    4,    4,    4,   

In [25]:
dataset = TensorDataset(input_tensor, target_tensor)
data_loader = DataLoader(dataset, batch_size=batch_size)

In [26]:
# input_dataset = TensorDataset(input_tensor)
# target_dataset = TensorDataset(target_tensor)

# # Create DataLoaders for input data and target data separately
# input_data_loader = DataLoader(input_dataset, batch_size=batch_size, shuffle=True)
# target_data_loader = DataLoader(target_dataset, batch_size=batch_size, shuffle=True)

In [27]:
print(len(input_tensor[0]))
len(target_tensor)

1000


40218

In [28]:
# import torch
# from torch.utils.data import TensorDataset, DataLoader
# from torch.nn.utils.rnn import pad_sequence

# def create_input_target_tensors(df, pad_token, batch_size):
#     input_tensors_batched = []
#     target_tensors_batched = []

#     # Iterate over batches
#     for batch_start in range(0, len(df), batch_size):
#         batch_df = df[batch_start:batch_start+batch_size]
        
#         # Find the length of the longest poem in the batch
#         max_length = max(batch_df['Token_Length'])-1
#         print(max_length)
#         # Initialize lists to store input and target tensors for the batch
#         input_tensors = []
#         target_tensors = []

#         input_tensors = [torch.tensor(poem) for poem in batch_df['Tokenized_Poem']]
#         print(len(input_tensors[0]))
#         padded_poem = torch.nn.utils.rnn.pad_sequence(input_tensors, batch_first=True, padding_value=pad_token)
#         print(len(padded_poem[0]))
#         # Iterate over poems in the batch
#         for poem in batch_df['Tokenized_Poem']:
#             # Pad tokens based on the length of the longest poem in the batch
#             padded_poem = torch.nn.utils.rnn.pad_sequence([poem], batch_first=True, padding_value=pad_token)
#             for i in range(len(poem) - 1):
#                 input_tensors.append(padded_poem[:, :i + 1])
#                 target_tensors.append(poem[i + 1])

#         # Concatenate input tensors for the batch
#         input_tensors_batched.append(torch.cat(input_tensors, dim=0))
#         target_tensors_batched.append(torch.tensor(target_tensors))

#     return input_tensors_batched, target_tensors_batched

# # Example usage
# pad_token = tokenizer.token_to_id("PAD")
# input_tensors_batched, target_tensors_batched = create_input_target_tensors(df, pad_token, batch_size)

# dataset = TensorDataset(input_tensors_batched, target_tensors_batched)
# data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)  

# # Verify shapes (optional)
# print(f"Number of batches: {len(input_tensors_batched)}")
# for i, (input_batch, target_batch) in enumerate(zip(input_tensors_batched, target_tensors_batched)):
#     print(f"Batch {i+1}: Input shape: {input_batch.shape}, Target shape: {target_batch.shape}")


In [29]:
# for i in data_loader:
#     print(i[0].shape)
#     break

In [30]:
pad_token

4

In [31]:
class WordEmbedder(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

    def forward(self, x):
        return self.embeddings(x)

In [32]:
# Now that we have successfully trained our word embedding layer, time to make our positional matrix, which we will make using the formula mentioned in the paper
def positional_encoding(seq_len, embedding_dim, device=device):
        positional_encoding = torch.zeros(seq_len, embedding_dim)
        position = torch.arange(0, seq_len, dtype=torch.float32).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dim, 2).float() * (-torch.log(torch.tensor(10000.0)) / embedding_dim))
        positional_encoding[:, 0::2] = torch.sin(position * div_term)
        positional_encoding[:, 1::2] = torch.cos(position * div_term)
        return positional_encoding

In [33]:
encoder_pos_encoding = positional_encoding(410, embedding_dim)
encoder_pos_encoding.shape

torch.Size([410, 64])

In [34]:
# Time to make the Multi-head Self Attention block
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, heads, embedding_dim):
        super(MultiHeadSelfAttention, self).__init__()
        self.heads = heads

        # 3 Linear Layers for Q, K and V
        self.w_q = nn.Linear(embedding_dim, embedding_dim)
        self.w_k = nn.Linear(embedding_dim, embedding_dim)
        self.w_v = nn.Linear(embedding_dim, embedding_dim)

        # Since the feature or embedding dimension is typically the last dimension
        self.softmax = nn.Softmax(dim=-1)

        # Last Linear layer for the attention
        self.w_a = nn.Linear(embedding_dim, embedding_dim)

    def forward(self, embedding_vector):
        batch_size, seq_len, embedding_dim = embedding_vector.size()
        # Compute Q, K, and V
        Q = self.w_q(embedding_vector)
        K = self.w_k(embedding_vector)
        V = self.w_v(embedding_vector)

        # Seperate into heads
        head_dim = embedding_dim // self.heads
        Q = Q.view(batch_size, self.heads, seq_len, head_dim)
        K = K.view(batch_size, self.heads, seq_len, head_dim)
        V = V.view(batch_size, self.heads, seq_len, head_dim)

        # Calculate attention
        attention = torch.matmul(self.softmax(torch.matmul(Q, K.transpose(-1, -2)) / torch.sqrt(torch.tensor(embedding_dim))), V)

        # Concatenating the attention heads (Transposing for correct concatenation)
        attention = attention.transpose(1, 2).reshape(batch_size, seq_len, embedding_dim)
        output = self.w_a(attention)
        return output

In [35]:
# Time to make the Masked Multi-head Self Attention block
class MaskedMultiHeadSelfAttention(nn.Module):
    def __init__(self, heads, embedding_dim):
        super(MaskedMultiHeadSelfAttention, self).__init__()
        self.heads = heads

        # 3 Linear Layers for Q, K and V
        self.w_q = nn.Linear(embedding_dim, embedding_dim)
        self.w_k = nn.Linear(embedding_dim, embedding_dim)
        self.w_v = nn.Linear(embedding_dim, embedding_dim)

        # Since the feature or embedding dimension is typically the last dimension
        self.softmax = nn.Softmax(dim=-1)

        # Last Linear layer for the attention
        self.w_a = nn.Linear(embedding_dim, embedding_dim)

    def forward(self, embedding_vector):
        batch_size, seq_len, embedding_dim = embedding_vector.size()
        # Compute Q, K, and V
        Q = self.w_q(embedding_vector)
        K = self.w_k(embedding_vector)
        V = self.w_v(embedding_vector)

        # Seperate into heads
        head_dim = embedding_dim // self.heads
        Q = Q.view(batch_size, self.heads, seq_len, head_dim)
        K = K.view(batch_size, self.heads, seq_len, head_dim)
        V = V.view(batch_size, self.heads, seq_len, head_dim)

        # Create a mask for masking the attention score
        mask = torch.triu(torch.ones(seq_len, seq_len, dtype=torch.bool), diagonal=1).unsqueeze(0).unsqueeze(0).expand(batch_size, self.heads, -1, -1).to(device)
        value_to_fill = float('-inf')

        # Calculate attention (including mask)
        attention = torch.matmul(self.softmax(torch.matmul(Q, K.transpose(-1, -2)).masked_fill(mask, value_to_fill) / torch.sqrt(torch.tensor(embedding_dim))), V)

        # Concatenating the attention heads (Transposing for correct concatenation)
        attention = attention.transpose(1, 2).reshape(batch_size, seq_len, embedding_dim)
        output = self.w_a(attention)
        return output

In [36]:
class AddNorm(nn.Module):
    def __init__(self, n_features):
        super(AddNorm, self).__init__()
        # Layer Norm will normalize the last dimension of the matrix
        self.norm = nn.LayerNorm(n_features)

    def forward(self, original, modified):
        return self.norm(original + modified)

In [37]:
class FeedForward(nn.Module):
    def __init__(self, embedding_dim):
        super(FeedForward, self).__init__()
        # Normally nn.Linear(embedding_dim, embedding_dim * 4) for expressiveness, we will change it if we have the resources to do so
        self.lr1 = nn.Linear(embedding_dim, embedding_dim)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(embedding_dim, embedding_dim)

    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        return x

In [38]:
# Time to build the Decoder
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, batch_size, max_length, heads):
        super(Decoder, self).__init__()
        self.word_embedder = WordEmbedder(vocab_size, embedding_dim)
        self.masked_attention = MaskedMultiHeadSelfAttention(heads, embedding_dim)
        self.add_norm1 = AddNorm(embedding_dim)
        self.attention = MultiHeadSelfAttention(heads, embedding_dim)
        self.add_norm2 = AddNorm(embedding_dim)
        self.feed_forward = FeedForward(embedding_dim)
        self.add_norm3 = AddNorm(embedding_dim)
        self.linear = nn.Linear(max_length * embedding_dim, vocab_size)

    def forward(self, x):
#         print(x.shape)
        x = self.word_embedder(x)
#         print(x.shape)
        x += positional_encoding(x.size(1), embedding_dim).unsqueeze(0).expand(x.size(0), -1, -1).to(device)
#         print(x.shape)
        x = self.add_norm1(x, self.masked_attention(x))
#         print(x.shape)
        x = self.add_norm2(x, self.attention(x))
#         print(x.shape)
        x = self.add_norm3(x, self.feed_forward(x))
#         print(x.shape)
        x = x.view(x.size(0), -1)
#         print(x.shape)
        logits = self.linear(x)
#         print(logits.shape)
        return F.softmax(logits, dim=-1)

In [39]:
model = Decoder(vocab_size, embedding_dim, batch_size, max_length, heads).to(device)

In [40]:
# Initialize our optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [41]:
def train_epoch(data_loader, model, optimizer, loss_fn, device, pad_token):
    model.train()
    total_loss = 0.0

    # Wrap your data loader with tqdm for a progress bar
    progress_bar = tqdm(data_loader, desc="Training", leave=True)

    for input_batch, target_batch in progress_bar:
        input_data = input_batch.to(device)  
        target_data = target_batch.to(device)
        
        target_one_hot = F.one_hot(target_data , num_classes=vocab_size).float()
        optimizer.zero_grad()
#         print(input_data.shape)
#         print(target_data.shape)
#         print(target_one_hot.shape)
#         mask = input_data.ne(pad_token)
        # Trim padded tokens based on the longest sequence in the batch
#         max_length = input_data.ne(pad_token).sum(1).max().item()
#         input_data = input_data[:, :max_length]
#         print(input_data.shape)
        # Forward pass
        logits = model(input_data)
#         print(logits.shape)
        # Calculate loss
        loss = loss_fn(logits, target_one_hot)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Update the progress bar with the current batch loss
        progress_bar.set_postfix(loss=loss.item())

    average_loss = total_loss / len(data_loader)
    return average_loss

# Example usage
for epoch in range(epochs):
    print(f"Starting Epoch {epoch + 1}/{epochs}")
    epoch_loss = train_epoch(data_loader, model, optimizer, loss_fn, device, pad_token)
    print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}")


Starting Epoch 1/15


Training: 100%|██████████| 2514/2514 [08:12<00:00,  5.10it/s, loss=9.21]


Epoch 1, Loss: 9.1617
Starting Epoch 2/15


Training: 100%|██████████| 2514/2514 [08:11<00:00,  5.11it/s, loss=9.21]


Epoch 2, Loss: 9.1615
Starting Epoch 3/15


...

Epoch 3, Loss: 9.1615
Starting Epoch 4/15


Training: 100%|██████████| 2514/2514 [08:14<00:00,  5.09it/s, loss=9.21]


Epoch 4, Loss: 9.1615
Starting Epoch 5/15


Training:  72%|███████▏  | 1814/2514 [05:56<02:17,  5.09it/s, loss=9.21]


KeyboardInterrupt: 

In [ ]:
model_path = "model_weights.pth"
torch.save(model.state_dict(), model_path)

In [ ]:
model.load_state_dict(torch.load("/kaggle/working/model_weights.pth"))
model.eval()

In [ ]:
def generate_poem(model, start_sequence, tokenizer=tokenizer, max_length=100):
    model.eval()  # Set the model to evaluation mode.
    tokens = tokenizer.encode(start_sequence)  # Encode the start sequence to tokens directly using the tokenizer.

    # Initialize the sequence with the token IDs of the start sequence
    generated_ids = tokens.ids

    for _ in range(max_length - len(generated_ids)):
        input_seq = torch.tensor([generated_ids], dtype=torch.long).to(device)
        with torch.no_grad():
            output_logits = model(input_seq)

        # Predict the next token ID (not index).
        next_token_id = output_logits[:, -1, :].argmax(-1).item()
        generated_ids.append(next_token_id)  # Append the ID of the next token.

        # Stop if the end token is generated.
        end_token_id = tokenizer.token_to_id("END")
        if next_token_id == end_token_id:
            break

    # Decode the token IDs back to text using the tokenizer.
    generated_text = tokenizer.decode(generated_ids)

    return generated_text

In [ ]:
start_sequence = "the tree"
poem = generate_poem(model, start_sequence)
print(poem)